In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies = movies.merge(credits,on="title")
movies.info()
movies = movies[['id','genres','keywords','original_language','original_title','overview','production_companies','cast','crew']]
movies.isnull().sum()
movies.dropna(inplace = True)
movies.duplicated().sum()
movies.iloc[0].genres

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [4]:
import ast
def converter(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l
movies['genres'] = movies['genres'].apply(converter)
movies['keywords'] = movies['keywords'].apply(converter)
movies['production_companies'] = movies['production_companies'].apply(converter)

In [5]:
def converter2(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
    return l
movies['crew'] = movies['crew'].apply(converter2)

In [6]:
import ast
def converter3(obj):
    l=[]
    c=0
    for i in ast.literal_eval(obj):
        if c<3:
            l.append(i['name'])
            c+=1
        else:
            break
    return l
movies['cast'] = movies['cast'].apply(converter3)
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [7]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])

movies['tags'] = movies['overview'] + movies['genres'] + movies['cast'] + movies['crew'] + movies['keywords'] ## concatinating columns

new_df = movies[['id','original_title','tags']]
##new_df = new_df.drop('original_title', axis=1) ## droping same name column

#new_df['tags'] = new_df['tags'].str.lower()# converting list items into lowercase

In [8]:

new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))
new_df.loc[:,'tags'] = new_df.loc[:,'tags'].str.lower()


C:\Users\SOUMITA\AppData\Local\Temp\ipykernel_4148\344326056.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [9]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [10]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\SOUMITA\AppData\Local\Temp\ipykernel_4148\1205700773.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [11]:
### Vectorization on tag column ####

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')
vectors = cv.fit_transform(new_df['tags']).toarray() ## converting into numpy array
feature = cv.get_feature_names_out()

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)
similarity[0] ## here similarity of 1st movie with other movies is compairing

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

In [16]:
def recommend(movie):
    movie_index = new_df[new_df['original_title'] == movie].index[0]
    distence = similarity[movie_index]
    movie_list = sorted(list(enumerate(distence)),reverse =True,key=lambda x:x[1])[1:6]

    for i in movie_list:
        print(new_df.iloc[i[0]].id)
        print(new_df.iloc[i[0]].original_title)
        print(new_df.iloc[i[0]].tags)
        

In [17]:
recommend('Avatar')

440
Aliens vs Predator: Requiem
a sequel to 2004' alien vs. predator, the icon creatur from two of the scariest film franchis in movi histori wage their most brutal battl ever - in our own backyard. the small town of gunnison, colorado becom a war zone between two of the deadliest extra-terrestri life form - the alien and the predator. when a predat scout ship crash-land in the hill outsid the town, alien facehugg and a hybrid alien/pred are releas and begin to terror the town. fantasi action sciencefict thriller horror stevenpasqual reikoaylesworth johnortiz colinstraus gregstraus predat nationalguard hybrid alien morgu alienpossess triangl infest xenomorph
679
Aliens
when ripley' lifepod is found by a salvag crew over 50 year later, she find that terra-form are on the veri planet they found the alien species. when the compani send a famili of colonist out to investig her story, all contact is lost with the planet and colonists. they enlist ripley and the coloni marin to return and se

In [14]:
import pickle
pickle.dump(new_df.to_dict(),open('movies_dictionary.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [15]:
new_df[new_df['original_title'] == 'Avatar'].index[0]

0